<a href="https://colab.research.google.com/github/Bergschaf/Ampfer_Mampfer_public/blob/main/Ampfer_Mampfer_Complete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Setup
%%bash
pip install haversine
pip install simplekml
pip install exif
git clone https://github.com/pytorch/vision.git
cd vision
git checkout v0.3.0
cp references/detection/utils.py ../
cp references/detection/transforms.py ../
cp references/detection/coco_eval.py ../
cp references/detection/engine.py ../
cp references/detection/coco_utils.py ../
cd .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for simplekml: filename=simplekml-1.3.6-py3-none-any.whl size=65876 sha256=2f1145163dac087dc7f632c99d275e461414ca5ceee88265a7434f63aea365b7
  Stored in directory: /root/.cache/pip/wheels/5c/ec/e6/10af1a1fb29ffca95151d4c886d6e06fc309c68f46519892de
Successfully built simplekml
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Cloning into 'vision'...
Note: checking out 'v0.3.0'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at be376084d version check against PyTorch's CUDA version


In [3]:
# Download Images
import os
if not os.path.exists("Bilder"):
  os.mkdir("Bilder") 
!gdown --folder https://drive.google.com/drive/folders/1U898P7M925KQ89KvOIjYJAwuzZxjRDtP?usp=sharing -q
os.chdir("/content")

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/requests/adapters.py", line 449, in send
    timeout=timeout
  File "/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py", line 600, in urlopen
    chunked=chunked)
  File "/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py", line 343, in _make_request
    self._validate_conn(conn)
  File "/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py", line 839, in _validate_conn
    conn.connect()
  File "/usr/local/lib/python3.7/dist-packages/urllib3/connection.py", line 332, in connect
    cert_reqs=resolve_cert_reqs(self.cert_reqs),
  File "/usr/local/lib/python3.7/dist-packages/urllib3/util/ssl_.py", line 272, in create_urllib3_context
    options |= OP_NO_SSLv2
  File "/usr/lib/python3.7/enum.py", line 838, in __or__
    result = self.__class__(self._value_ | self.__class__(other)._value_)
  File "/usr/lib/python3.7/enum.py", line 315, in __call__
    return cls.__new__

In [5]:
# Download Networks
import os
if not os.path.exists("Networks"):
  os.mkdir("Networks") 
os.chdir("Networks")
!gdown 1k3Yjk6Emb_ePUgzQIfSdJq8jxgRoNpg4
os.chdir("/content")

Downloading...
From: https://drive.google.com/uc?id=1k3Yjk6Emb_ePUgzQIfSdJq8jxgRoNpg4
To: /content/Networks/model_drohne_niedrig_V2.pt
100% 166M/166M [00:00<00:00, 266MB/s]


In [ ]:
# Ermitteln der Koordinaten der Ampfer
import torch
import torch.utils.data
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from engine import evaluate
from PIL import Image
import utils
import transforms as T
import os
import math
import haversine
from haversine import inverse_haversine
from exif import Image as EImage
import json
import simplekml


if not os.path.exists("Koordinaten"):
  os.mkdir("Koordinaten")
bilderp = "Bilder/"  # Ordner, in den die Bilder hochgeladen wurden
targetp = "Koordinaten/"  # Ordner, in den die ermittelten Koordinaten gespeichert werden


class Dataloader(torch.utils.data.Dataset):
    def __init__(self, transforms=None):
        self.Bilder = bilderp
        self.pdir = sorted(os.listdir(self.Bilder))
        self.Data = {}
        self.Boxes = []
        self.Transforms = transforms

    def __getitem__(self, idx):
        """
        Zum Trainieren des Netzes soll diese Funktion, die Label des Bildes an der Position idx
        zurückgeben. Da hier das Netz nur angewendet und nicht trainiert wird, werden diese Label
        nicht benötigt. Anstelle der Label wird daher der Dateiname des Bildes zurückgegeben, sodass
        man die erkannten Ampfer später den Metadaten des Bildes zuordnen kann.
        """
        img = Image.open(self.Bilder + self.pdir[idx]).convert("RGB")
        filename = {"filename": self.pdir[idx]}
        if self.Transforms is not None:
            img, filename = self.Transforms(img, filename)
        return img, filename

    def __len__(self):
        return int(len(self.pdir))


def get_model(num_classes):
    """
    Diese Funktion ist dafür zuständig, das Netz zu laden.
    """
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)  # Das pretrained-Model von pytorch
    in_features = model.roi_heads.box_predictor.cls_score.in_features  # wird heruntergeladen und konfiguriert.
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    
    if os.path.isfile("Networks/model_drohne_niedrig_V2.pt"):
        print("---loaded model---")
        model.load_state_dict(torch.load("Networks/model_drohne_niedrig_V2.pt"))
    else:
        print("---model not found---")
    # Wenn bereits ein trainiertes Netz vorhanden ist, dann werden die entsprechenden Gewichtungen geladen:
    return model


class Img:
    """
    Die Img Klasse wird für Bilder erstellt, um die Überlappung der Boxen, die die Ampferpflanzen markieren, zu
    entfernen.
    """
    def __init__(self, filename: str, objects):
        self.filename = filename  # Dateiname des Bildes (ohne Dateiendung)
        self.objects = objects  # Die erkannten Positionen der Ampferpflanzen.

    def remove_overlap(self):
        """
        Diese Funktion überprüft, ob sich die Boxen überlappen.
        """
        for index, object in enumerate(self.objects):
            for index2, object2 in enumerate(self.objects[index:]):
                # Das Programm überorüft für jede Kombination von zwei Boxen, ob sie sich überlappen.
                overlap, smaller = check_overlap(object, object2)  # Die check_overlap-Funktion gibt zurück, zu wie groß
                # die sich überlappende Fläche im Verlgeich zur Fläche der kleineren Box ist (in Prozent).
                if overlap > 0.8:  # Wenn sich die Boxen zu mehr als 80 % überlappen, dann wird die kleinere gelöscht.
                    try:
                        if smaller == 1:  # Die smaller Variable gibt an, welche Box kleiner ist.
                            self.objects.remove(object)
                        else:
                            self.objects.remove(object2)
                    except ValueError:  # Wenn die Box schon gelöscht wurde, dann wird nichts gemacht.
                        pass


def check_overlap(pos1, pos2):
    """
    Die check_overlap-Funktion gibt zurück, wie viel sich die zwei Boxen pos1 und pos2 überlappen.
    """
    difx = 0  # Die überlappung in x-Richtung.
    if pos2[0] < pos1[0] < pos2[2]:
        difx = pos2[2] - pos1[0]
        if pos2[0] < pos1[2] < pos2[2]:
            difx = pos2[2] - pos2[0]

    elif pos1[0] < pos2[0] < pos1[2]:
        difx = pos1[2] - pos2[0]
        if pos1[0] < pos2[2] < pos1[2]:
            difx = pos1[2] - pos1[0]

    dify = 0  # Die Überlappung in y-Richtung.
    if pos2[1] < pos1[1] < pos2[3]:
        dify = pos2[3] - pos1[1]
        if pos2[1] < pos1[3] < pos2[3]:
            dify = pos2[3] - pos2[1]

    elif pos1[1] < pos2[1] < pos1[3]:
        dify = pos1[3] - pos2[1]
        if pos1[1] < pos2[3] < pos1[3]:
            dify = pos1[3] - pos1[1]

    overlap = difx * dify  # Die Fläche der Überlappung.

    A_pos1 = (pos1[2] - pos1[0]) * (pos1[3] - pos1[1])  # Die Fläche von Box 1.
    A_pos2 = (pos2[2] - pos2[0]) * (pos2[3] - pos2[1])  # Die Fläche von Box 2.
    # Das Verhältniss der Fläche der Überlappung und der Fläche der kleineren Box wird in Prozent zurückgegeben.
    return (overlap / A_pos1, 1) if A_pos1 < A_pos2 else (overlap / A_pos2, 2)


def decimal_coords(coords, ref):
    """
    Diese Funktion konvertiert die Koorinaten coords (mit der Referenz ref) in Dezimalkoordinaten.
    """
    decimal_degrees = coords[0] + coords[1] / 60 + coords[2] / 3600
    if ref == "S" or ref == "W":
        decimal_degrees = -decimal_degrees
    return decimal_degrees


def get_flight_yaw_altitude_coords(image_path):
    """
    Diese Funktion liest die Höhe, die Koordinaten und den Winkel nach Norden der Drohen aus den Metadaten des Bildes
    an der Position image_path aus.
    """
    with open(image_path, "rb") as img:
        img_text = img.read()
        image = EImage(img_text)
        coords = (decimal_coords(image.gps_latitude,
                                 image.gps_latitude_ref),
                  decimal_coords(image.gps_longitude,
                                 image.gps_longitude_ref))
        # Die Koordinaten können aus den exif-Metadaten der Bilder ausgelesen werden.
        start_pos = img_text.find(b"FlightYawDegree=") + 17  # Der Winkel nach Norden und die Höhe der Drohne können
        # einfach aus den Bytes des Bildes ausgelesen werden.
        for i, b in enumerate(img_text[start_pos:]):
            if b == 34:
                end_pos = i
                break
        yaw = float(img_text[start_pos:end_pos + start_pos])
        start_pos = img_text.find(b"RelativeAltitude=") + 18
        for i, b in enumerate(img_text[start_pos:]):
            if b == 34:
                end_pos = i
                break
        alt = float(img_text[start_pos:end_pos + start_pos])
        return yaw, alt, coords


def get_Angle_from_Camera(x: int, y: int):
    """
    Diese Funktion gibt die Distanz des Bildpunktes x y zum Bildmittelpunkt und den Winkel zur y-Achse zurück.
    """
    new_x = x - 2000
    new_y = y - 1500
    dist = 0
    if new_x != 0 and new_y != 0:
        dist = math.sqrt(pow(new_x, 2) + pow(new_y, 2))
    return dist, math.atan2(y - 1500, x - 2000) + math.pi / 2


def get_Ampfer_Cords(filename, boxes, Img_dir):
    """
    Diese Funktion gibt eine Liste an Koordinaten zurück, an denen die Ampfer sind, deren Position auf dem Bild in der
    boxes Liste gespeichert ist.
    """
    IMG_PATH = f"{Img_dir}{filename}.JPG"
    ampfer_coords = []
    yaw, alt, coords = get_flight_yaw_altitude_coords(IMG_PATH)  # Die Metadaten werden ausgelesen.
    meter_per_pixel = (alt * math.tan(math.radians(39.5))) / 2500  # Ein Umrechnugsfaktor, wie viele Pixel im Bild
    # einem Meter entsprechen, wird berechnet.
    for x, y in boxes:
        dist_pixel, angle_from_middle = get_Angle_from_Camera(x, y)
        abs_angle = math.radians(yaw) + angle_from_middle  # Die Distanz zum Bildmittelpunkt und der Winkel nach
        dist = dist_pixel * meter_per_pixel  # Norden wird für jeden Ampfer berechnet.
        ampfer_coords.append(inverse_haversine(coords, dist, abs_angle, unit=haversine.Unit.METERS))
        # Dann werden die Koordinaten des Ampfers berechnet und an die amper_coords-Liste angehängt.
    return ampfer_coords


if __name__ == "__main__":
    dataset = Dataloader(transforms=T.Compose([T.ToTensor()]))
    torch.manual_seed(1)
    indices = torch.randperm(len(dataset)).tolist()
    dataset = torch.utils.data.Subset(dataset, indices)
    data_loader = torch.utils.data.DataLoader(
        Dataloader(transforms=T.Compose(T.ToTensor())), batch_size=2, shuffle=False, num_workers=2,
        collate_fn=utils.collate_fn)
    loaded_model = get_model(num_classes=2).eval()
    # Zuerst wird das Netz und der Dataloader initialisiert.

    for idx in range(len(os.listdir(bilderp))):

        img, filename = dataset[idx]

        with torch.no_grad():
            prediction = loaded_model([img])  # Das Netz wird dann auf jedes Bild angewendet.
        objects = []
        pic_name = filename["filename"]
        for element in range(len(prediction[0]["boxes"])):
            boxes = prediction[0]["boxes"][element].cpu().numpy()
            pos = (int(boxes[0]), int(boxes[1]), int(boxes[2]), int(boxes[3]))
            objects.append(pos)
        # Dann wird die boxes-Liste anhand der Netzausgabe erstellt.
        img = Img(pic_name[:-4], objects)
        img.remove_overlap()  # Mit der remove_overlap-Funtkion der Image-Klasse werden dann die Überlappungen entfernt.
        boxes = []
        for box in img.objects:
            boxes.append(((box[0] + box[2]) / 2, (box[1] + box[3]) / 2))
        cordslist = get_Ampfer_Cords(pic_name[:-4], boxes, bilderp)  # Anhand der Bildkoordinaten des Ampfers werden
        # dann die realen Koordinaten des Ampfers berechet.
        json.dumps(cordslist)
        with open(targetp + pic_name[:-4] + ".json", "w") as f:
            f.write(json.dumps(cordslist))
        kml = simplekml.Kml()
        for i in cordslist:
            kml.newpoint(name="Ampfer", coords=[(i[1], i[0])])
        kml.save(f"{targetp}/{pic_name[:-4]}.kml")
        # Diese Koordinaten werden dann in einer .json und in einer .kml Datei gespeichert.
        # .kml Dateien können in Google maps geöffnet werden
        print(pic_name + " done!")

---loaded model---
131.JPG done!
139.JPG done!
128.JPG done!
137.JPG done!


IsADirectoryError: ignored

In [7]:
# Einzeichnen der Ampferpositionen auf den Bildern
import logging
import numpy as np
import torch
import torch.utils.data
import PIL
from PIL import Image, ImageDraw, ImageFont
import pandas as pd
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from engine import train_one_epoch, evaluate
import utils
import transforms as T
import xml.etree.ElementTree as ET
import os
import time

bilderp = "Bilder/"
if not os.path.exists("Output"):
  os.mkdir("Output")
outputp = "Output/"


class Dataloader(torch.utils.data.Dataset):
    def __init__(self, transforms=None):
        self.Bilder = bilderp
        self.pdir = sorted(os.listdir(self.Bilder))
        print(self.pdir)
        self.Data = {}
        self.Boxes = []
        self.Transforms = transforms

    def __getitem__(self, idx):  # returns a dictionary with filenames as keys
        # only returns the labeled pictures as a list: [Boxes,RGB-Image]

        img = Image.open(self.Bilder + self.pdir[idx]).convert("RGB")
        print(self.pdir[idx])

        target = {"filename":self.pdir[idx]}
        if self.Transforms is not None:
            img, target = self.Transforms(img, target)
        return img, target

    def __len__(self):
        return int(len(self.pdir))


def get_model(num_classes):
    # load an object detection model pre-trained on COCO
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    # get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new on
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    if os.path.isfile("Networks/model_drohne_niedrig_V2.pt"):
        print("---loaded model---")
        model.load_state_dict(torch.load("Networks/model_drohne_niedrig_V2.pt"))
    else:
        logging.warning("model not found")
    return model


def get_transform(train):
    transforms = []
    transforms.append(T.ToTensor())
    if train:
        # during training, randomly flip the training images
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)


dataset = Dataloader(transforms=get_transform(train=False))
torch.manual_seed(1)
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices)
data_loader = torch.utils.data.DataLoader(
    Dataloader(transforms=get_transform(train=False)), batch_size=2, shuffle=False, num_workers=2,
    collate_fn=utils.collate_fn)

loaded_model = get_model(num_classes=2)

for idx in range(len(os.listdir(bilderp))):

  img, target = dataset[idx]
  
  #put the model in evaluation mode
  loaded_model.eval()
  with torch.no_grad():
    prediction = loaded_model([img])

  image = Image.fromarray(img.mul(255).permute(1, 2,0).byte().numpy())
  draw = ImageDraw.Draw(image)


  for element in range(len(prediction[0]["boxes"])):
    boxes = prediction[0]["boxes"][element].cpu().numpy()
    score = np.round(prediction[0]["scores"][element].cpu().numpy(),
                      decimals= 4)
    if True:
        draw.rectangle([(boxes[0], boxes[1]), (boxes[2], boxes[3])], 
        outline ="red", width =10)
        fnt = ImageFont.truetype("usr/share/fronts/truetype/liberation/LiberationMono-Regular.ttf",  size=100)
        draw.text((boxes[0]+10, boxes[1]+10), text = str(score), fill=(255,255,255,255))
  image.save(outputp + str(target["filename"])[:-4] + ".JPG")

['126.JPG', '127.JPG', '128.JPG', '129.JPG', '130.JPG', '131.JPG', '132.JPG', '133.JPG', '134.JPG', '97.JPG', '98.JPG', '99.JPG', 'Bilder']
['126.JPG', '127.JPG', '128.JPG', '129.JPG', '130.JPG', '131.JPG', '132.JPG', '133.JPG', '134.JPG', '97.JPG', '98.JPG', '99.JPG', 'Bilder']


/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


---loaded model---
132.JPG


IsADirectoryError: ignored